# Hybrid recommender

### Importing github repository

In [1]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
token = user_secrets.get_secret("token")

!git clone https://{token}@github.com/Benedart/RecSys-2022-Challenge-Polimi.git

fatal: destination path 'RecSys-2022-Challenge-Polimi' already exists and is not an empty directory.


In [2]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
token = user_secrets.get_secret("token")

import os

os.chdir( "./RecSys-2022-Challenge-Polimi")

!git pull https://{token}@github.com/Benedart/RecSys-2022-Challenge-Polimi.git

remote: Enumerating objects: 105, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 105 (delta 27), reused 41 (delta 18), pack-reused 54
Receiving objects: 100% (105/105), 113.39 MiB | 53.58 MiB/s, done.
Resolving deltas: 100% (50/50), completed with 6 local objects.
From https://github.com/Benedart/RecSys-2022-Challenge-Polimi
 * branch            HEAD       -> FETCH_HEAD
Updating 23642d5..5b4c427
Fast-forward
 HyperparameterTuning/SearchAbstractClass.py        |   2 +-
 HyperparameterTuning/run_hyperparameter_search.py  |   4 +-
 .../Cython/MatrixFactorization_Cython.py           |  35 +-
 .../Cython/MatrixFactorization_Cython_Epoch.pyx    |  96 ++++-
 Recommenders/Recommender_import_list.py            |   2 +-
 Recommenders/SLIM/Cython/Triangular_Matrix.pyx     |  24 +-
 challenge_notebooks/A look at ImpressionList.ipynb | 206 +++++++++
 .../Classes to Identify Noise.ipynb                | 471 +++++++++++++++++++++
 ..

#### Compiling Cython files

In [ ]:
import os

os.chdir( "./RecSys-2022-Challenge-Polimi")

In [ ]:
import os

os.chdir( "./RecSys-2022-Challenge-Polimi")
!python run_compile_all_cython.py

In [ ]:
!cp -r /kaggle/input/recsys-models-optimization/result_experiments/SKOPT_test/ result_experiments/SKOPT_test/

os.chdir("result_experiments/SKOPT_test")
! for i in */; do (cd "$i"; zip -r "../${i%/}.zip" .); done
! rm -R -- */
os.chdir("../..")

In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy.sparse as sps
import csv

from Recommenders.Recommender_import_list import *

#### Importing URM & ICM

In [5]:
URM_all = pd.read_csv("/kaggle/input/recommender-system-2022-challenge-polimi/interactions_and_impressions.csv")
ICM_type = pd.read_csv("/kaggle/input/recommender-system-2022-challenge-polimi/data_ICM_type.csv")
ICM_length = pd.read_csv("/kaggle/input/recommender-system-2022-challenge-polimi/data_ICM_length.csv")

ICM_all = pd.concat([ICM_type, ICM_length]).sort_values(by='item_id')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
URM_all

In [ ]:
ICM_all

#### Getting rid of duplicates and empty indices
Let's start with the indices, the mapping is done for consistency

In [6]:
mapped_id, original_id = pd.factorize(URM_all["UserID"].unique())
user_original_Id_to_index = pd.Series(mapped_id, index=original_id)

print("Unique UserID in the URM are {}".format(len(original_id)))

mapped_id, original_id = pd.factorize(URM_all["ItemID"].unique())

print("Unique ItemID in the URM are {}".format(len(original_id)))

all_item_indices = pd.concat([URM_all["ItemID"], ICM_all["item_id"]], ignore_index=True)
mapped_id, original_id = pd.factorize(all_item_indices.unique())

print("Unique ItemID in the URM and ICM are {}".format(len(original_id)))

item_original_ID_to_index = pd.Series(mapped_id, index=original_id)

mapped_id, original_id = pd.factorize(ICM_all["feature_id"].unique())
feature_original_ID_to_index = pd.Series(mapped_id, index=original_id)

print("Unique FeatureID in the URM are {}".format(len(feature_original_ID_to_index)))

URM_all["UserID"] = URM_all["UserID"].map(user_original_Id_to_index)
URM_all["ItemID"] = URM_all["ItemID"].map(item_original_ID_to_index)
ICM_all["item_id"] = ICM_all["item_id"].map(item_original_ID_to_index)
ICM_all["feature_id"] = ICM_all["feature_id"].map(feature_original_ID_to_index)

Unique UserID in the URM are 41629
Unique ItemID in the URM are 24507
Unique ItemID in the URM and ICM are 27968
Unique FeatureID in the URM are 6


And then we remove duplicate values

In [7]:
URM_all = URM_all.drop(["Impressions", "Data"], axis=1).drop_duplicates()
URM_all["Rating"] = 1

In [ ]:
URM_all

In [ ]:
ICM_all

#### Creating the matrices

In [8]:
n_users = len(user_original_Id_to_index)
n_items = len(item_original_ID_to_index)
n_features = len(feature_original_ID_to_index)

from sklearn.model_selection import train_test_split

seed = 42

(user_ids_training_validation, user_ids_test,
 item_ids_training_validation, item_ids_test,
 ratings_training_validation, ratings_test) = train_test_split(URM_all.UserID,
                                                    URM_all.ItemID,
                                                    URM_all.Rating,
                                                    test_size=0.15,
                                                    shuffle=True,
                                                    random_state=seed)

(user_ids_training, user_ids_validation,
 item_ids_training, item_ids_validation,
 ratings_training, ratings_validation) = train_test_split(user_ids_training_validation,
                                                          item_ids_training_validation,
                                                          ratings_training_validation,
                                                          test_size=0.15,
                                                          shuffle=True,
                                                          random_state=seed)

URM_all = sps.csr_matrix((URM_all.Rating, (URM_all.UserID, URM_all.ItemID)),
                           shape = (n_users, n_items))

URM_train = sps.csr_matrix((ratings_training, (user_ids_training, item_ids_training)),
                           shape = (n_users, n_items))

URM_validation = sps.csr_matrix((ratings_validation, (user_ids_validation, item_ids_validation)),
                           shape = (n_users, n_items))

URM_train_validation = sps.csr_matrix((ratings_training_validation, (user_ids_training_validation, item_ids_training_validation)),
                           shape = (n_users, n_items))

URM_test = sps.csr_matrix((ratings_test, (user_ids_test, item_ids_test)),
                           shape = (n_users, n_items))


ICM_all = sps.csr_matrix((np.ones(len(ICM_all["item_id"].values)), 
                          (ICM_all["item_id"].values, ICM_all["feature_id"].values)),
                        shape = (n_items, n_features))

ICM_all.data = np.ones_like(ICM_all.data)

ICM_all = sps.csr_matrix(ICM_all)
features_per_item = np.ediff1d(ICM_all.indptr)

ICM_all = sps.csc_matrix(ICM_all)
items_per_feature = np.ediff1d(ICM_all.indptr)

ICM_all = sps.csr_matrix(ICM_all)

#### Setting up evaluators

In [9]:
from Evaluation.Evaluator import EvaluatorHoldout

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 1520 ( 3.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 976 ( 2.3%) Users that have less than 1 test interactions


#### Creating SLIM_Elastic model

In [28]:
ItemKNN_CFCBF = ItemKNN_CFCBF_Hybrid_Recommender(URM_train, ICM_all)
ItemKNN_CFCBF.fit(
    topK= 40, 
    shrink= 34, 
    similarity= 'tversky', 
    normalize= True, 
    tversky_alpha= 0.1911656636857864, 
    tversky_beta= 1.4823556453405908, 
    ICM_weight= 0.026917911342859663
)

ItemKNN_CFCBF_HybridRecommender: URM Detected 3461 (12.4%) items with no interactions.
ItemKNN_CFCBF_HybridRecommender: ICM Detected 4877 (17.4%) items with no features.
Similarity column 27968 (100.0%), 1689.36 column/sec. Elapsed time 16.56 sec


In [ ]:
evaluator_test.evaluateRecommender(ItemKNN_CFCBF)

In [29]:
UserKNN = UserKNNCFRecommender(URM_train)
UserKNN.fit(
    topK= 836, 
    shrink= 0, 
    similarity= 'tversky', 
    normalize= True, 
    tversky_alpha= 1.7489968882972766, 
    tversky_beta= 3.0
)

UserKNNCFRecommender: URM Detected 3461 (12.4%) items with no interactions.
Similarity column 41629 (100.0%), 2166.79 column/sec. Elapsed time 19.21 sec


In [ ]:
evaluator_test.evaluateRecommender(UserKNN)

In [8]:
SLIM_Elastic_old = SLIMElasticNetRecommender(URM_train_validation)
""" 
# this was the best before the reproducible splits
SLIM_Elastic.fit(
    topK= 2298, 
    l1_ratio= 0.006495366789349948, 
    alpha= 0.001
)

SLIM_Elastic.fit(
    topK=5325, 
    l1_ratio= 1e-05, 
    alpha= 0.001
)
"""

SLIM_Elastic_old.fit(
    topK= 10000, 
    l1_ratio= 0.03255609147342309, 
    alpha= 0.001
)

SLIMElasticNetRecommender: URM Detected 3461 (12.4%) items with no interactions.
SLIMElasticNetRecommender: Processed 3161 (11.3%) in 5.00 min. Items per second: 10.53
SLIMElasticNetRecommender: Processed 6669 (23.8%) in 10.00 min. Items per second: 11.11
SLIMElasticNetRecommender: Processed 10402 (37.2%) in 15.00 min. Items per second: 11.56
SLIMElasticNetRecommender: Processed 14126 (50.5%) in 20.00 min. Items per second: 11.77
SLIMElasticNetRecommender: Processed 17919 (64.1%) in 25.00 min. Items per second: 11.94
SLIMElasticNetRecommender: Processed 21791 (77.9%) in 30.00 min. Items per second: 12.10
SLIMElasticNetRecommender: Processed 24638 (88.1%) in 35.01 min. Items per second: 11.73
SLIMElasticNetRecommender: Processed 25067 (89.6%) in 40.01 min. Items per second: 10.44
SLIMElasticNetRecommender: Processed 25494 (91.2%) in 45.02 min. Items per second: 9.44
SLIMElasticNetRecommender: Processed 25928 (92.7%) in 50.03 min. Items per second: 8.64
SLIMElasticNetRecommender: Process

In [10]:
SLIM_Elastic = SLIMElasticNetRecommender(URM_train_validation)
SLIM_Elastic.load_model("result_experiments/SKOPT_test", SLIM_Elastic.RECOMMENDER_NAME + "_best_model_last.zip")

SLIMElasticNetRecommender: URM Detected 3461 (12.4%) items with no interactions.
SLIMElasticNetRecommender: Loading model from file 'result_experiments/SKOPT_testSLIMElasticNetRecommender_best_model_last.zip'
SLIMElasticNetRecommender: Loading complete


In [11]:
evaluator_test.evaluateRecommender(SLIM_Elastic)

EvaluatorHoldout: Processed 40653 (100.0%) in 56.87 sec. Users per second: 715


(       PRECISION PRECISION_RECALL_MIN_DEN    RECALL       MAP MAP_MIN_DEN  \
 cutoff                                                                      
 10      0.054163                 0.090007  0.084078  0.026964    0.043834   
 
              MRR      NDCG        F1  HIT_RATE ARHR_ALL_HITS  ...  \
 cutoff                                                        ...   
 10      0.179425  0.087077  0.065884  0.355226      0.218413  ...   
 
        COVERAGE_USER COVERAGE_USER_HIT USERS_IN_GT DIVERSITY_GINI  \
 cutoff                                                              
 10          0.976555          0.346898    0.976555       0.014255   
 
        SHANNON_ENTROPY RATIO_DIVERSITY_HERFINDAHL RATIO_DIVERSITY_GINI  \
 cutoff                                                                   
 10            8.541745                   0.991672             0.031982   
 
        RATIO_SHANNON_ENTROPY RATIO_AVERAGE_POPULARITY RATIO_NOVELTY  
 cutoff                                   

In [8]:
SLIM_Elastic = SLIMElasticNetRecommender(URM_train_validation)

SLIM_Elastic.fit(
    topK= 6468, 
    l1_ratio= 0.03534306561986866, 
    alpha= 0.001
)

evaluator_test.evaluateRecommender(SLIM_Elastic)

SLIMElasticNetRecommender: URM Detected 3461 (12.4%) items with no interactions.
SLIMElasticNetRecommender: Processed 2574 ( 9.2%) in 5.00 min. Items per second: 8.58
SLIMElasticNetRecommender: Processed 5355 (19.1%) in 10.00 min. Items per second: 8.92
SLIMElasticNetRecommender: Processed 8390 (30.0%) in 15.00 min. Items per second: 9.32
SLIMElasticNetRecommender: Processed 11451 (40.9%) in 20.00 min. Items per second: 9.54
SLIMElasticNetRecommender: Processed 14534 (52.0%) in 25.00 min. Items per second: 9.69
SLIMElasticNetRecommender: Processed 17663 (63.2%) in 30.01 min. Items per second: 9.81
SLIMElasticNetRecommender: Processed 20691 (74.0%) in 35.01 min. Items per second: 9.85
SLIMElasticNetRecommender: Processed 23882 (85.4%) in 40.01 min. Items per second: 9.95
SLIMElasticNetRecommender: Processed 24786 (88.6%) in 45.02 min. Items per second: 9.18
SLIMElasticNetRecommender: Processed 25138 (89.9%) in 50.03 min. Items per second: 8.37
SLIMElasticNetRecommender: Processed 25491 

(       PRECISION PRECISION_RECALL_MIN_DEN    RECALL       MAP MAP_MIN_DEN  \
 cutoff                                                                      
 10      0.055804                 0.093011  0.086899  0.027753    0.045197   
 
              MRR      NDCG        F1  HIT_RATE ARHR_ALL_HITS  ...  \
 cutoff                                                        ...   
 10      0.184063  0.089693  0.067964  0.364303      0.224396  ...   
 
        COVERAGE_USER COVERAGE_USER_HIT USERS_IN_GT DIVERSITY_GINI  \
 cutoff                                                              
 10          0.976555          0.355762    0.976555       0.021256   
 
        SHANNON_ENTROPY RATIO_DIVERSITY_HERFINDAHL RATIO_DIVERSITY_GINI  \
 cutoff                                                                   
 10            9.103578                   0.993872             0.047689   
 
        RATIO_SHANNON_ENTROPY RATIO_AVERAGE_POPULARITY RATIO_NOVELTY  
 cutoff                                   

In [10]:
# this one should be fine
SLIM_Elastic = SLIMElasticNetRecommender(URM_train_validation)
SLIM_Elastic.load_model("models", "SLIM_Elastic_train_validation.zip")

evaluator_test.evaluateRecommender(SLIM_Elastic)

SLIMElasticNetRecommender: URM Detected 3461 (12.4%) items with no interactions.
SLIMElasticNetRecommender: Loading model from file 'modelsSLIM_Elastic_train_validation.zip'
SLIMElasticNetRecommender: Loading complete


ValueError: dimension mismatch

In [11]:
RP3Beta = RP3betaRecommender(URM_train_validation)
""" # possibly overfitted
RP3Beta.fit(
    topK= 87,
    alpha= 0.4251768169663159, 
    beta= 0.2480238726919643,
    normalize_similarity= True
)

RP3Beta.fit(
    topK= 135,
    alpha= 0.8473968835576251, 
    beta= 0.25814662527577076,
    normalize_similarity= True
)
"""

RP3Beta.fit(**{
    "topK": 63,
    "alpha": 0.6871585126786798,
    "beta": 0.2893401754537936,
    "normalize_similarity": True
})

RP3betaRecommender: URM Detected 3461 (12.4%) items with no interactions.
RP3betaRecommender: Similarity column 27968 (100.0%), 2142.05 column/sec. Elapsed time 13.06 sec


In [12]:
evaluator_test.evaluateRecommender(RP3Beta)

EvaluatorHoldout: Processed 40653 (100.0%) in 33.69 sec. Users per second: 1207


(       PRECISION PRECISION_RECALL_MIN_DEN    RECALL       MAP MAP_MIN_DEN  \
 cutoff                                                                      
 10      0.052601                 0.087559  0.081734  0.025692    0.041978   
 
              MRR      NDCG        F1  HIT_RATE ARHR_ALL_HITS  ...  \
 cutoff                                                        ...   
 10      0.172248  0.083956  0.064009  0.348289      0.208826  ...   
 
        COVERAGE_USER COVERAGE_USER_HIT USERS_IN_GT DIVERSITY_GINI  \
 cutoff                                                              
 10          0.976555          0.340123    0.976555       0.064687   
 
        SHANNON_ENTROPY RATIO_DIVERSITY_HERFINDAHL RATIO_DIVERSITY_GINI  \
 cutoff                                                                   
 10           10.005459                   0.996481             0.145131   
 
        RATIO_SHANNON_ENTROPY RATIO_AVERAGE_POPULARITY RATIO_NOVELTY  
 cutoff                                   

In [ ]:
PureSVD = PureSVDRecommender(URM_train)
PureSVD.fit(num_factors=19)

In [ ]:
evaluator_test.evaluateRecommender(PureSVD)

### Hybrid recommender
#### Defining the hybrid classes

In [10]:
from Recommenders.BaseRecommender import BaseRecommender

class ScoresHybridRecommender(BaseRecommender):
    """ ScoresHybridRecommender
    Hybrid of two prediction scores R = R1*alpha + R2*(1-alpha)

    """

    RECOMMENDER_NAME = "ScoresHybridRecommender"

    def __init__(self, URM_train, recommender_1, recommender_2):
        super(ScoresHybridRecommender, self).__init__(URM_train)

        self.URM_train = sps.csr_matrix(URM_train)
        self.recommender_1 = recommender_1
        self.recommender_2 = recommender_2
        
        
    def fit(self, alpha = 0.5):
        self.alpha = alpha      


    def _compute_item_score(self, user_id_array, items_to_compute):
        
        # In a simple extension this could be a loop over a list of pretrained recommender objects
        item_weights_1 = self.recommender_1._compute_item_score(user_id_array)
        item_weights_2 = self.recommender_2._compute_item_score(user_id_array)

        item_weights = item_weights_1*self.alpha + item_weights_2*(1-self.alpha)

        return item_weights

    

class DifferentLossScoresHybridRecommender(BaseRecommender):
    """ ScoresHybridRecommender
    Hybrid of two prediction scores R = R1/norm*alpha + R2/norm*(1-alpha) where R1 and R2 come from
    algorithms trained on different loss functions.

    """

    RECOMMENDER_NAME = "DifferentLossScoresHybridRecommender"


    def __init__(self, URM_train, recommender_1, recommender_2):
        super(DifferentLossScoresHybridRecommender, self).__init__(URM_train)

        self.URM_train = sps.csr_matrix(URM_train)
        self.recommender_1 = recommender_1
        self.recommender_2 = recommender_2
        
        
        
    def fit(self, norm, alpha = 0.5):

        self.alpha = alpha
        self.norm = norm


    def _compute_item_score(self, user_id_array, items_to_compute):
        
        item_weights_1 = self.recommender_1._compute_item_score(user_id_array)
        item_weights_2 = self.recommender_2._compute_item_score(user_id_array)

        norm_item_weights_1 = LA.norm(item_weights_1, self.norm)
        norm_item_weights_2 = LA.norm(item_weights_2, self.norm)
        
        
        if norm_item_weights_1 == 0:
            raise ValueError("Norm {} of item weights for recommender 1 is zero. Avoiding division by zero".format(self.norm))
        
        if norm_item_weights_2 == 0:
            raise ValueError("Norm {} of item weights for recommender 2 is zero. Avoiding division by zero".format(self.norm))
        
        item_weights = item_weights_1 / norm_item_weights_1 * self.alpha + item_weights_2 / norm_item_weights_2 * (1-self.alpha)

        return item_weights
    
class CustomHybridRecommender(BaseRecommender):

    RECOMMENDER_NAME = "CustomHybridRecommender"

    def __init__(self, URM_train, verbose=True):
        super(CustomHybridRecommender, self).__init__(URM_train)
        
        self.SLIM_Elastic = SLIMElasticNetRecommender(URM_train)
        self.SLIM_Elastic.load_model("result_experiments/SKOPT_test", SLIM_Elastic.RECOMMENDER_NAME + "_best_model_last.zip")
        
        self.ItemKNN_CFCBF = ItemKNN_CFCBF_Hybrid_Recommender(URM_train, ICM_all)
        self.ItemKNN_CFCBF.fit(
            topK= 40, 
            shrink= 34, 
            similarity= 'tversky', 
            normalize= True, 
            tversky_alpha= 0.1911656636857864, 
            tversky_beta= 1.4823556453405908, 
            ICM_weight= 0.026917911342859663
        )
        
        self.RP3Beta = RP3betaRecommender(URM_train)
        self.RP3Beta.fit(
            topK= 87,
            alpha= 0.4251768169663159, 
            beta= 0.2480238726919643,
            normalize_similarity= True
        )
        
        """
        self.UserKNN = UserKNNCFRecommender(URM_train)
        self.UserKNN.fit(
            topK= 836, 
            shrink= 0, 
            similarity= 'tversky', 
            normalize= True, 
            tversky_alpha= 1.7489968882972766, 
            tversky_beta= 3.0
        )
        """

    def fit(self, SLIM_Elastic_weight, ItemKNNCFCBF_weight, RP3Beta_weight):
        self.SLIM_Elastic_weight = SLIM_Elastic_weight
        self.ItemKNNCFCBF_weight = ItemKNNCFCBF_weight
        self.RP3Beta_weight = RP3Beta_weight

    def save_model(self, folder_path, file_name=None):
        pass

    def _compute_item_score(self, user_id_array, items_to_compute=None):
        result = self.SLIM_Elastic_weight*self.SLIM_Elastic._compute_item_score(user_id_array,items_to_compute)
        result += self.ItemKNNCFCBF_weight*self.ItemKNN_CFCBF._compute_item_score(user_id_array,items_to_compute)
        result += self.UserKNN_weight*self.UserKNN._compute_item_score(user_id_array,items_to_compute)
   #     for index in range(1,len(self.alphas)):
   #         result = result + self.alphas[index]*self.recommenders[index]._compute_item_score(user_id_array,items_to_compute)
        return result

In [11]:
from Recommenders.BaseRecommender import BaseRecommender

class GeneralizedMergedHybridRecommender(BaseRecommender):
    """
    This recommender merges two recommendes by weighting their ratings
    """

    RECOMMENDER_NAME = "GeneralizedMergedHybridRecommender"

    def __init__(self, URM_train, recommenders: list, verbose=True):
        self.RECOMMENDER_NAME = ''
        for recommender in recommenders:
            self.RECOMMENDER_NAME = self.RECOMMENDER_NAME + recommender.RECOMMENDER_NAME[:-11]
        self.RECOMMENDER_NAME = self.RECOMMENDER_NAME + 'HybridRecommender'

        super(GeneralizedMergedHybridRecommender, self).__init__(
            URM_train,
            verbose=verbose
        )

        self.recommenders = recommenders

    def fit(self, SLIM_Elastic_weight, RP3Beta_weight, ItemKNNCFCBF_weight, UserKNN_weight):
        self.SLIM_Elastic_weight = SLIM_Elastic_weight
        self.RP3Beta_weight = RP3Beta_weight
        self.ItemKNNCFCBF_weight = ItemKNNCFCBF_weight
        self.UserKNN_weight = UserKNN_weight

    def save_model(self, folder_path, file_name=None):
        pass

    def _compute_item_score(self, user_id_array, items_to_compute=None):
        result = self.SLIM_Elastic_weight*self.recommenders[0]._compute_item_score(user_id_array,items_to_compute)
        result += self.RP3Beta_weight*self.recommenders[1]._compute_item_score(user_id_array,items_to_compute)
        result += self.ItemKNNCFCBF_weight*self.recommenders[2]._compute_item_score(user_id_array,items_to_compute)
        result += self.UserKNN_weight*self.recommenders[3]._compute_item_score(user_id_array,items_to_compute)
   #     for index in range(1,len(self.alphas)):
   #         result = result + self.alphas[index]*self.recommenders[index]._compute_item_score(user_id_array,items_to_compute)
        return result

#### Creating the hybrid model

In [ ]:
user_id = 42

In [ ]:
RP3Beta_score = RP3Beta._compute_item_score(user_id) 
RP3Beta_score

In [ ]:
SLIM_Elastic_score = SLIM_Elastic._compute_item_score(user_id)
SLIM_Elastic_score

In [42]:
from skopt.space import Real, Integer, Categorical
import traceback

from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from HyperparameterTuning.SearchSingleCase import SearchSingleCase
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs

def runHyperparameterSearch_customHybrid(recommender_class, URM_train, recommenders, URM_train_last_test = None,
                                          n_cases = None, n_random_starts = None, resume_from_saved = False,
                                          save_model = "best", evaluate_on_test = "best", max_total_time = None,
                                          evaluator_validation = None, evaluator_test = None, evaluator_validation_earlystopping = None,
                                          metric_to_optimize = None, cutoff_to_optimize = None,
                                          output_folder_path ="result_experiments/", parallelizeKNN = True,
                                          allow_weighting = True, allow_bias_URM=False, allow_dropout_MF = False, similarity_type_list = None):
    """
    This function performs the hyperparameter optimization for a collaborative recommender
    :param recommender_class:   Class of the recommender object to optimize, it must be a BaseRecommender type
    :param URM_train:           Sparse matrix containing the URM training data
    :param URM_train_last_test: Sparse matrix containing the union of URM training and validation data to be used in the last evaluation
    :param n_cases:             Number of hyperparameter sets to explore
    :param n_random_starts:     Number of the initial random hyperparameter values to explore, usually set at 30% of n_cases
    :param resume_from_saved:   Boolean value, if True the optimization is resumed from the saved files, if False a new one is done
    :param save_model:          ["no", "best", "last"] which of the models to save, see HyperparameterTuning/SearchAbstractClass for details
    :param evaluate_on_test:    ["all", "best", "last", "no"] when to evaluate the model on the test data, see HyperparameterTuning/SearchAbstractClass for details
    :param max_total_time:    [None or int] if set stops the hyperparameter optimization when the time in seconds for training and validation exceeds the threshold
    :param evaluator_validation:    Evaluator object to be used for the validation of each hyperparameter set
    :param evaluator_validation_earlystopping:   Evaluator object to be used for the earlystopping of ML algorithms, can be the same of evaluator_validation
    :param evaluator_test:          Evaluator object to be used for the test results, the output will only be saved but not used
    :param metric_to_optimize:  String with the name of the metric to be optimized as contained in the output of the evaluator objects
    :param cutoff_to_optimize:  Integer with the recommendation list length to be optimized as contained in the output of the evaluator objects
    :param output_folder_path:  Folder in which to save the output files
    :param parallelizeKNN:      Boolean value, if True the various heuristics of the KNNs will be computed in parallel, if False sequentially
    :param allow_weighting:     Boolean value, if True it enables the use of TF-IDF and BM25 to weight features, users and items in KNNs
    :param allow_bias_URM:      Boolean value, if True it enables the use of bias to shift the values of the URM
    :param allow_dropout_MF:    Boolean value, if True it enables the use of dropout on the latent factors of MF algorithms
    :param similarity_type_list: List of strings with the similarity heuristics to be used for the KNNs
    """



    # If directory does not exist, create
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)

    earlystopping_keywargs = {"validation_every_n": 5,
                              "stop_on_validation": True,
                              "evaluator_object": evaluator_validation_earlystopping,
                              "lower_validations_allowed": 5,
                              "validation_metric": metric_to_optimize,
                              }

    URM_train = URM_train.copy()
    recommenders = recommenders.copy()

    n_users, n_items = URM_train.shape

    if URM_train_last_test is not None:
        URM_train_last_test = URM_train_last_test.copy()

    try:

        output_file_name_root = recommender_class.RECOMMENDER_NAME

        hyperparameterSearch = SearchBayesianSkopt(recommender_class, evaluator_validation=evaluator_validation, evaluator_test=evaluator_test)
    
       ##########################################################################################################

        if recommender_class is GeneralizedMergedHybridRecommender:

            hyperparameters_range_dictionary = {
                "SLIM_Elastic_weight": Real(low = 0, high = 2, prior = 'uniform'),
                "RP3Beta_weight": Real(low = 0, high = 2, prior = 'uniform'),
                "ItemKNNCFCBF_weight": Real(low = 0, high = 2, prior = 'uniform'),
                "UserKNN_weight": Real(low = 0, high = 2, prior = 'uniform'),
            }

            recommender_input_args = SearchInputRecommenderArgs(
                CONSTRUCTOR_POSITIONAL_ARGS = [URM_train, recommenders],
                CONSTRUCTOR_KEYWORD_ARGS = {},
                FIT_POSITIONAL_ARGS = [],
                FIT_KEYWORD_ARGS = {},
                EARLYSTOPPING_KEYWORD_ARGS = {},
            )

       #########################################################################################################

        if URM_train_last_test is not None:
            recommender_input_args_last_test = recommender_input_args.copy()
            recommender_input_args_last_test.CONSTRUCTOR_POSITIONAL_ARGS[0] = URM_train_last_test
        else:
            recommender_input_args_last_test = None

        ## Final step, after the hyperparameter range has been defined for each type of algorithm
        hyperparameterSearch.search(recommender_input_args,
                               hyperparameter_search_space= hyperparameters_range_dictionary,
                               n_cases = n_cases,
                               n_random_starts = n_random_starts,
                               resume_from_saved = resume_from_saved,
                               save_model = save_model,
                               evaluate_on_test = evaluate_on_test,
                               max_total_time = max_total_time,
                               output_folder_path = output_folder_path,
                               output_file_name_root = output_file_name_root,
                               metric_to_optimize = metric_to_optimize,
                               cutoff_to_optimize = cutoff_to_optimize,
                               recommender_input_args_last_test = recommender_input_args_last_test)




    except Exception as e:

        print("On recommender {} Exception {}".format(recommender_class, str(e)))
        traceback.print_exc()

        error_file = open(output_folder_path + "ErrorLog.txt", "a")
        error_file.write("On recommender {} Exception {}\n".format(recommender_class, str(e)))
        error_file.close()

In [ ]:
recommenders = [SLIM_Elastic, RP3Beta, ItemKNN_CFCBF, UserKNN] # ORDER MATTER!!!!!!!!!

from functools import partial

output_folder_path = "result_experiments/SKOPT_test/"

n_cases = 300
n_random_starts = int(n_cases * 0.25)

runHyperparameterSearch_customHybrid_partial = partial(
                    runHyperparameterSearch_customHybrid,
                    URM_train = URM_train,
                    recommenders = recommenders,
                    URM_train_last_test = URM_train_validation,
                    metric_to_optimize = "MAP",
                    cutoff_to_optimize = 10,
                    n_cases = n_cases,
                    n_random_starts = n_random_starts,
                    evaluator_validation_earlystopping = evaluator_validation,
                    evaluator_validation = evaluator_validation,
                    evaluator_test = evaluator_test,
                    output_folder_path = output_folder_path,
                    resume_from_saved = False
)

runHyperparameterSearch_customHybrid_partial(GeneralizedMergedHybridRecommender)

In [ ]:
!zip -r hybrid_results_Elastic_RP3_ItemKNN_UserKNN.zip result_experiments/SKOPT_test/GeneralizedMergedHybridRecommender*

In [ ]:
hybrid_recommender = Generalized(URM_train_validation)

In [ ]:
hybrid_recommender = GeneralizedMergedHybridRecommender(
    URM_train = URM_train_validation, 
    recommenders=recommenders
)

hybrid_recommender.fit(2.0, 0.1767812598481143, 2.0)

print(recommenders)

In [ ]:
evaluator_test.evaluateRecommender(hybrid_recommender)

<hr>

## Random tries

In [21]:
IALS = IALSRecommender(URM_train_validation)
IALS.fit(**{
    'epochs': 34,
    'num_factors': 79,
    'alpha': 5.743821416717141,
    'epsilon': 0.3498236011181738,
    'reg': 0.13292125016403122,
    "validation_every_n": 5,
    "stop_on_validation": True,
    "evaluator_object": evaluator_validation,
    "lower_validations_allowed": 5,
    "validation_metric": "MAP",
})

evaluator_test.evaluateRecommender(IALS)

IALSRecommender: URM Detected 3461 (12.4%) items with no interactions.
IALSRecommender: Epoch 1 of 34. Elapsed time 32.62 sec
IALSRecommender: Epoch 2 of 34. Elapsed time 1.09 min
IALSRecommender: Epoch 3 of 34. Elapsed time 1.65 min
IALSRecommender: Epoch 4 of 34. Elapsed time 2.19 min
IALSRecommender: Validation begins...
EvaluatorHoldout: Processed 40109 (100.0%) in 40.08 sec. Users per second: 1001
IALSRecommender: CUTOFF: 10 - PRECISION: 0.0000000, PRECISION_RECALL_MIN_DEN: 0.0000000, RECALL: 0.0000000, MAP: 0.0000000, MAP_MIN_DEN: 0.0000000, MRR: 0.0000000, NDCG: 0.0000000, F1: 0.0000000, HIT_RATE: 0.0000000, ARHR_ALL_HITS: 0.0000000, NOVELTY: 0.0037333, AVERAGE_POPULARITY: 0.1604166, DIVERSITY_MEAN_INTER_LIST: 0.9638425, DIVERSITY_HERFINDAHL: 0.9963819, COVERAGE_ITEM: 0.0445509, COVERAGE_ITEM_HIT: 0.0000000, ITEMS_IN_GT: 0.8628432, COVERAGE_USER: 0.9634870, COVERAGE_USER_HIT: 0.0000000, USERS_IN_GT: 0.9634870, DIVERSITY_GINI: 0.0108419, SHANNON_ENTROPY: 8.6671033, RATIO_DIVERSIT

(       PRECISION PRECISION_RECALL_MIN_DEN    RECALL       MAP MAP_MIN_DEN  \
 cutoff                                                                      
 10      0.041225                 0.069101  0.064768  0.019182     0.03199   
 
              MRR      NDCG        F1  HIT_RATE ARHR_ALL_HITS  ...  \
 cutoff                                                        ...   
 10      0.137902  0.065683  0.050382  0.293582      0.161689  ...   
 
        COVERAGE_USER COVERAGE_USER_HIT USERS_IN_GT DIVERSITY_GINI  \
 cutoff                                                              
 10          0.976555          0.286699    0.976555       0.010835   
 
        SHANNON_ENTROPY RATIO_DIVERSITY_HERFINDAHL RATIO_DIVERSITY_GINI  \
 cutoff                                                                   
 10            8.666144                   0.996708             0.024309   
 
        RATIO_SHANNON_ENTROPY RATIO_AVERAGE_POPULARITY RATIO_NOVELTY  
 cutoff                                   

In [18]:
IALS.save_model(".", "IALS_train_validation")

IALSRecommender: Saving model in file '.IALS_train_validation'
IALSRecommender: Saving complete


In [13]:
recommenders = [SLIM_Elastic, RP3Beta] 
alphas = [2.097339118738173, 1.0568674680305168]

recommender = GeneralizedLinearHybridRecommender(
    URM_train = URM_train_validation, 
    recommenders=recommenders
)

recommender.fit(alphas)

SLIMElasticNetRP3betaHybridRecommender: URM Detected 3461 (12.4%) items with no interactions.


In [14]:
evaluator_test.evaluateRecommender(recommender) #27923

EvaluatorHoldout: Processed 40653 (100.0%) in 1.07 min. Users per second: 631


(       PRECISION PRECISION_RECALL_MIN_DEN    RECALL       MAP MAP_MIN_DEN  \
 cutoff                                                                      
 10      0.056146                 0.093555  0.087395  0.027923    0.045523   
 
              MRR      NDCG        F1 HIT_RATE ARHR_ALL_HITS  ...  \
 cutoff                                                       ...   
 10      0.184495  0.090174  0.068369  0.36536      0.225285  ...   
 
        COVERAGE_USER COVERAGE_USER_HIT USERS_IN_GT DIVERSITY_GINI  \
 cutoff                                                              
 10          0.976555          0.356795    0.976555       0.022732   
 
        SHANNON_ENTROPY RATIO_DIVERSITY_HERFINDAHL RATIO_DIVERSITY_GINI  \
 cutoff                                                                   
 10            9.224434                   0.994627             0.051001   
 
        RATIO_SHANNON_ENTROPY RATIO_AVERAGE_POPULARITY RATIO_NOVELTY  
 cutoff                                      

In [14]:
new_alphas = [3, 1.25]

recommender = GeneralizedLinearHybridRecommender(
    URM_train = URM_train_validation, 
    recommenders=recommenders
)

recommender.fit(new_alphas)

evaluator_test.evaluateRecommender(recommender)

SLIMElasticNetRP3betaHybridRecommender: URM Detected 3461 (12.4%) items with no interactions.
EvaluatorHoldout: Processed 40653 (100.0%) in 1.03 min. Users per second: 659


(       PRECISION PRECISION_RECALL_MIN_DEN    RECALL       MAP MAP_MIN_DEN  \
 cutoff                                                                      
 10       0.05618                 0.093531  0.087365  0.027924    0.045486   
 
              MRR      NDCG        F1  HIT_RATE ARHR_ALL_HITS  ...  \
 cutoff                                                        ...   
 10      0.184404  0.090145  0.068385  0.365164      0.225252  ...   
 
        COVERAGE_USER COVERAGE_USER_HIT USERS_IN_GT DIVERSITY_GINI  \
 cutoff                                                              
 10          0.976555          0.356602    0.976555       0.022805   
 
        SHANNON_ENTROPY RATIO_DIVERSITY_HERFINDAHL RATIO_DIVERSITY_GINI  \
 cutoff                                                                   
 10             9.23073                   0.994625             0.051166   
 
        RATIO_SHANNON_ENTROPY RATIO_AVERAGE_POPULARITY RATIO_NOVELTY  
 cutoff                                   

|Alphas|Score|
|------|-----|
|[2, 1.75]|0.027859|
|[2, 1.6]|0.027853|
|[2, 1.4]|0.027853|
|**[2, 1.5]**|**0.0279307**|
|**[2, 0.7]**|**0.0279307**|
|[2, 0.75]|0.027928|
|[2, 0.8]|0.027925|
|[2, 0.69]|0.027923|
|[2, 0.6]|0.027921|
|[1.8, 0.8]|0.027916|
|[2, 0.5]|0.027905|
|[2.5, 0.7]|0.027913|
|[2.1, 0.7]|0.027919|
|[2.05, 0.7]|0.02792|
|[1.95, 0.7]|0.027928|